# Visualizando a COVID-19 no Nordeste
> Evolução e estágio atual dos estados e municípios nordestinos

- toc: true 
- badges: true
- hide_binder_badge: true
- comments: true
- categories: [colab]
- image: images/nordeste.png

> Caso esteja visualizando este post em um dispositivo móvel, por favor utilize seu dispositivo na horizontal.

Este post reúne análises das ocorrências de COVID-19 nos estados do Nordeste. Os dados utilizados para análise foram coletados pelo projeto [Brasil.io](https://brasil.io) dos sites das secretarias de saúde dos estados e municípios brasileiros.

A análise inicial é feita a nível de estados, comparando suas situações com a situação do Brasil. Esta análise apresenta diferentes métricas, tanto absolutas (confirmados e mortes) quanto relativas (confirmados e mortes por 100 mil habitantes). Além disso, apresentamos a análise por dia e em dias decorridos desde o primeiro caso.

> Sempre que alternar entre tipos de condição (confirmado ou morte), clique duas vezes na área principal do gráfico para que o zoom se ajuste (a área principal do gráfico tem pano de fundo quadriculado).

A análise final é feita a nível de municípios e usa georeferenciamento para visualizar a trajetória do vírus pelo Nordeste. No mapa, é possível visualizar informações sobre cada localidade passando o mouse por cima do município desejado.

> É possível animar a linha do tempo da evolução dos casos apertando o botão play, ou visualizar uma data específica usando o slider.

Em cada gráfico, várias outras opções de interação estão disponíveis, podendo ser exploradas pelo menu que aparece no canto superior direito de cada um.

In [1]:
#hide
!pip install plotly==4.5.4

     |████████████████████████████████| 7.1MB 3.3MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [0]:
#hide
import requests
import datetime

import folium
import pandas as pd
import numpy as np
import plotly.express as px

#### Evolução e estágio atual nos estados

In [0]:
#hide
page = 1
base_url = url = "https://brasil.io/api/dataset/covid19/caso/data?format=json&page=" 
df_list = []

while True:
    json_brazil = requests.get(f"{url}{page}").json()
    if "count" not in json_brazil:
        break
    df_list.append(pd.DataFrame(json_brazil["results"]))
    page += 1

df = pd.concat(df_list)

In [0]:
#hide
def reindex_by_date(df, max_day):
    min_day = min(df["date"])
    df = df.set_index("date")
    dates = pd.date_range(min_day, max_day, name="date")
    return df.reindex(dates).ffill()

In [0]:
#hide
ne_states = ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"]
df_states = df.query(f"place_type == 'state' and state in {ne_states}")
df_states = df_states.drop(["city","is_last","order_for_place","place_type"], axis=1)

last_day = max(df_states["date"])
df_states["date"] = pd.to_datetime(df_states["date"])
df_states = df_states.groupby("state").apply(lambda df: reindex_by_date(df, last_day))
df_states = df_states.reset_index("date").sort_values("date")

df_states.columns = ["Data", "Código IBGE", "Confirmado", "Confirmado / cem mil", "Letalidade", "Morte", "População", "UF"]
df_states["Morte / cem mil"] = df_states["Morte"] / df_states["População"] * 100000
df_states = df_states.drop(["Letalidade", "População"], axis=1)

ts_states = df_states.melt(id_vars=["Código IBGE", "Data", "UF"], var_name="Métrica", value_name="Valor")

##### Casos por dia

In [10]:
#hide_input
po = px.line(ts_states, x="Data", y="Valor", color="UF", animation_frame="Métrica")
po.update_layout(
#    title='Evolução dos casos de COVID-19 no Brasil',
    xaxis_title="Dia",
    yaxis_title="Métrica",
    updatemenus=[{"visible": False}],
)

##### Casos em dias desde o primeiro caso confirmado

In [0]:
#hide
df_states["Data"] = pd.to_datetime(df_states["Data"])
start_date = df_states.groupby("UF")["Data"].min().reset_index()
start_date.columns = ["UF", "Dia0"]

# adding day zero to df
df_day0 = pd.merge(start_date, df_states)
days_elapsed = pd.to_datetime(df_day0["Data"]) - pd.to_datetime(df_day0["Dia0"])
df_day0["Dias"] = days_elapsed.astype('timedelta64[D]')

ts_day0 = df_day0.drop(["Código IBGE", "Data", "Dia0"], axis=1).melt(id_vars=["UF", "Dias"], var_name="Condição", value_name="Quantidade")

In [12]:
#hide_input
po = px.line(ts_day0, x="Dias", y="Quantidade", color="UF", animation_frame="Condição")
po.update_layout(
#    title='Evolução dos casos de COVID-19 no Brasil',
    xaxis_title="Dias desde o primeiro caso confirmado",
    yaxis_title="Casos confirmados",
    updatemenus=[{"visible": False}],
)

#### Trajetória de dispersão pelo Nordeste

In [91]:
#hide
df_cities = df.query(f"place_type == 'city' and state in {ne_states}")
df_cities = df_cities.drop(["is_last","order_for_place","place_type"], axis=1).dropna(subset=["city_ibge_code"])
df_cities.head()

,city,city_ibge_code,confirmed,confirmed_per_100k_inhabitants,date,death_rate,deaths,estimated_population_2019,state
7,Maceió,2704302,13,1.27583,2020-03-31,0.0769,1.0,1018948.0,AL
8,Porto Real do Colégio,2707503,1,4.98355,2020-03-31,NaN,0.0,20066.0,AL
22,Alagoinhas,2900702,3,1.97894,2020-03-31,NaN,0.0,151596.0,BA
23,Barra,2902708,1,1.86644,2020-03-31,NaN,0.0,53578.0,BA
24,Barreiras,2903201,1,0.64334,2020-03-31,NaN,0.0,155439.0,BA


In [92]:
df_aggregated = df_cities.pivot_table(index=["date", "city", "city_ibge_code", "estimated_population_2019", "state"], 
                                      values=["confirmed", "confirmed_per_100k_inhabitants", "deaths"], 
                                      aggfunc='sum').reset_index()
df_aggregated.head()

,date,city,city_ibge_code,estimated_population_2019,state,confirmed,confirmed_per_100k_inhabitants,deaths
0,2020-03-06,Feira de Santana,2910800,614872.0,BA,1,0.16264,0.0
1,2020-03-07,Feira de Santana,2910800,614872.0,BA,2,0.32527,0.0
2,2020-03-08,Maceió,2704302,1018948.0,AL,1,0.09814,0.0
3,2020-03-09,Feira de Santana,2910800,614872.0,BA,2,0.32527,0.0
4,2020-03-09,Maceió,2704302,1018948.0,AL,1,0.09814,0.0


In [0]:
last_day = max(df_aggregated["date"])
df_aggregated["date"] = pd.to_datetime(df_aggregated["date"])
df_aggregated = df_aggregated.groupby("city").apply(lambda df: reindex_by_date(df, last_day))
df_aggregated = df_aggregated.reset_index("date").sort_values("date")

df_aggregated.columns = ["Data", "Município", "Código IBGE", "População", "UF", "Confirmado", "Confirmado / cem mil", "Morte"]
df_aggregated["Morte / cem mil"] = df_aggregated["Morte"] / df_aggregated["População"] * 100000
df_aggregated = df_aggregated.drop(["População"], axis=1)
df_aggregated["log_confirmed"] = np.log1p(df_aggregated["Confirmado / cem mil"])
df_aggregated["Data"] = df_aggregated["Data"].dt.strftime("%d-%m")

In [0]:
#hide
ne = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/2/?formato=application/vnd.geo+json&resolucao=5").json()
cities = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/regioes/2/distritos").json()
for city in ne['features']:
    city['id'] = city['properties']['codarea']

In [94]:
cities_with_cases = set(df_aggregated.query("Confirmado > 0")["Código IBGE"].unique())
len(cities_with_cases)

84

In [95]:
features = [city for city in ne["features"] if city["id"] in cities_with_cases]
len(features)

84

In [0]:
ne["features"] = features

In [97]:
#hide_input

fig = px.choropleth(df_aggregated,
                    geojson=ne, color="log_confirmed", 
                    locations="Código IBGE", hover_name="Município",
                    hover_data=["UF", "Confirmado", "Confirmado / cem mil", "Morte", "Morte / cem mil"],
                    animation_frame="Data",
                    center={"lat": -5.78362, "lon": -36.914},
                    range_color=(0, max(df_aggregated["log_confirmed"])),
                    color_continuous_scale='reds',
                    labels = {
                              "log_confirmed": "Casos confirmados (em escala logarítmica)"
                              }
                    )

fig.update_geos(projection={"scale": 30}, 
                fitbounds="locations", 
                visible=False,
                )

fig.layout.coloraxis.colorbar.showticklabels = False
fig.layout.coloraxis.colorbar.title = "Máximo"

fig.show()